In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import * 

spark = SparkSession.builder.appName("Read The Data").config("spark.some.config.option","some-value").getOrCreate()

In [3]:
purchaseSchema = StructType([
    StructField("Date", DateType(), True),
    StructField("Time", StringType(), True),
    StructField("City", StringType(), True),
    StructField("Item", StringType(), True),
    StructField("Total", FloatType(), True),
    StructField("Payment", StringType(), True),
])  

In [4]:
purchaseDataframe = spark.read.csv(
    "dataset/purchases.csv", 
    header=True, schema=purchaseSchema, sep="\t")
purchaseDataframe.show(3)

+----------+-----+----------+----------------+------+-------+
|      Date| Time|      City|            Item| Total|Payment|
+----------+-----+----------+----------------+------+-------+
|2012-01-01|09:00|  San Jose|  Men's Clothing|214.05|   Amex|
|2012-01-01|09:00|Fort Worth|Women's Clothing|153.57|   Visa|
|2012-01-01|09:00| San Diego|           Music| 66.08|   Cash|
+----------+-----+----------+----------------+------+-------+
only showing top 3 rows



In [5]:
#count number of rows of our dataFrame
num_rows = purchaseDataframe.count()
print("number of rows: ", num_rows)
#show our dataFrame schema
purchaseDataframe.printSchema()
#show statistic of the data we want
purchaseDataframe.describe('Total').show()

number of rows:  4138476
root
 |-- Date: date (nullable = true)
 |-- Time: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Item: string (nullable = true)
 |-- Total: float (nullable = true)
 |-- Payment: string (nullable = true)

+-------+------------------+
|summary|             Total|
+-------+------------------+
|  count|           4138476|
|   mean| 249.9610854939852|
| stddev|144.31741115429654|
|    min|               0.0|
|    max|            499.99|
+-------+------------------+



In [6]:
#create new dataFrame from "City" and "Total" columns
newDataframe = purchaseDataframe.select(purchaseDataframe['City'], 
                                              purchaseDataframe['Total'])
newDataframe.show(3); 
newDataframe.printSchema() 

+----------+------+
|      City| Total|
+----------+------+
|  San Jose|214.05|
|Fort Worth|153.57|
| San Diego| 66.08|
+----------+------+
only showing top 3 rows

root
 |-- City: string (nullable = true)
 |-- Total: float (nullable = true)



In [7]:

#adding constant value of 10 to every row data in "Total" column
purchaseDataframe.select(purchaseDataframe['City'],
                         purchaseDataframe['Total']+10).show(3)

+----------+------------+
|      City|(Total + 10)|
+----------+------------+
|  San Jose|      224.05|
|Fort Worth|      163.57|
| San Diego|       76.08|
+----------+------------+
only showing top 3 rows



In [8]:
#filter only row data whose "Total" column value > 200
purchaseDataframe.filter(purchaseDataframe['Total'] > 200).show(3)

+----------+-----+----------+-------------------+------+----------+
|      Date| Time|      City|               Item| Total|   Payment|
+----------+-----+----------+-------------------+------+----------+
|2012-01-01|09:00|  San Jose|     Men's Clothing|214.05|      Amex|
|2012-01-01|09:00|Pittsburgh|       Pet Supplies|493.51|  Discover|
|2012-01-01|09:00|     Omaha|Children's Clothing|235.63|MasterCard|
+----------+-----+----------+-------------------+------+----------+
only showing top 3 rows



In [9]:
sortedByCity = purchaseDataframe.orderBy('City').show(4)

+----------+-----+-----------+-----------------+------+----------+
|      Date| Time|       City|             Item| Total|   Payment|
+----------+-----+-----------+-----------------+------+----------+
|2012-11-22|14:58|Albuquerque|             Toys|154.55|MasterCard|
|2012-11-22|15:00|Albuquerque| Women's Clothing|349.43|      Cash|
|2012-11-22|15:00|Albuquerque|             Toys|499.27|MasterCard|
|2012-11-22|15:19|Albuquerque|Health and Beauty| 15.76|      Visa|
+----------+-----+-----------+-----------------+------+----------+
only showing top 4 rows



In [10]:
numTransactionEachCity = purchaseDataframe.groupBy("City").count()
numTransactionEachCity.show(5)

+---------------+-----+
|           City|count|
+---------------+-----+
|North Las Vegas|40013|
|        Phoenix|40333|
|          Omaha|40209|
|      Anchorage|39806|
|        Anaheim|40086|
+---------------+-----+
only showing top 5 rows



In [11]:
from pyspark.sql.functions import monotonically_increasing_id

newPurchasedDataframe = purchaseDataframe.withColumn(
    "index", monotonically_increasing_id())
newPurchasedDataframe.show(7)
row2Till4 = newPurchasedDataframe.filter((newPurchasedDataframe['index']>=2) &
                                         (newPurchasedDataframe['index']<=4))
row2Till4.show()

+----------+-----+----------+-------------------+------+----------+-----+
|      Date| Time|      City|               Item| Total|   Payment|index|
+----------+-----+----------+-------------------+------+----------+-----+
|2012-01-01|09:00|  San Jose|     Men's Clothing|214.05|      Amex|    0|
|2012-01-01|09:00|Fort Worth|   Women's Clothing|153.57|      Visa|    1|
|2012-01-01|09:00| San Diego|              Music| 66.08|      Cash|    2|
|2012-01-01|09:00|Pittsburgh|       Pet Supplies|493.51|  Discover|    3|
|2012-01-01|09:00|     Omaha|Children's Clothing|235.63|MasterCard|    4|
|2012-01-01|09:00|  Stockton|     Men's Clothing|247.18|MasterCard|    5|
|2012-01-01|09:00|    Austin|            Cameras| 379.6|      Visa|    6|
+----------+-----+----------+-------------------+------+----------+-----+
only showing top 7 rows

+----------+-----+----------+-------------------+------+----------+-----+
|      Date| Time|      City|               Item| Total|   Payment|index|
+----------+-

In [12]:
dataRow2ColumnTotal = newPurchasedDataframe.filter(newPurchasedDataframe['index']==2).select('Total')
dataRow2ColumnTotal.show()

+-----+
|Total|
+-----+
|66.08|
+-----+



In [13]:
#we need to make sql temporary view for our dataFrame
purchaseDataframe.createOrReplaceTempView("purchaseSql")

#select "Total" dan "Payment" column from our sql temporary view
anotherNewDataframe = spark.sql("SELECT Total, Payment FROM purchaseSql")
anotherNewDataframe.show(3)

+------+-------+
| Total|Payment|
+------+-------+
|214.05|   Amex|
|153.57|   Visa|
| 66.08|   Cash|
+------+-------+
only showing top 3 rows



In [14]:
#sorting data by "City" column alphabetically
orderByCity = spark.sql("SELECT * FROM purchaseSql ORDER BY City")
orderByCity.show(5)

+----------+-----+-----------+-----------------+------+----------+
|      Date| Time|       City|             Item| Total|   Payment|
+----------+-----+-----------+-----------------+------+----------+
|2012-11-22|15:00|Albuquerque| Women's Clothing|349.43|      Cash|
|2012-11-22|15:00|Albuquerque|             Toys|499.27|MasterCard|
|2012-11-22|14:58|Albuquerque|             Toys|154.55|MasterCard|
|2012-11-22|15:19|Albuquerque|Health and Beauty| 15.76|      Visa|
|2012-11-22|15:20|Albuquerque|   Sporting Goods|386.11|      Cash|
+----------+-----+-----------+-----------------+------+----------+
only showing top 5 rows



In [15]:
filterAndSortWithSQL = spark.sql("SELECT * FROM purchaseSql WHERE Total>200 ORDER BY Payment")
filterAndSortWithSQL.show(4)

+----------+-----+---------------+-----------------+------+-------+
|      Date| Time|           City|             Item| Total|Payment|
+----------+-----+---------------+-----------------+------+-------+
|2012-11-22|14:59|       Columbus|Health and Beauty|431.92|   Amex|
|2012-11-22|15:00|           Reno|           Garden|293.73|   Amex|
|2012-11-22|14:59|     Sacramento|     Pet Supplies|495.36|   Amex|
|2012-11-22|14:58|North Las Vegas|            Music| 219.7|   Amex|
+----------+-----+---------------+-----------------+------+-------+
only showing top 4 rows

